In [ ]:
#G050.2213-00.6063
#19:25:57.52 15:03:00.3
#3.3 kpc
#Band 6 ALMA
#CH3OH (methanol)

In [ ]:
# For Colab
!pip install peakutils
!pip install lineid_plot
!pip install astroquery
!pip install lmfit
!wget https://github.com/saint-germain/rot_diag/raw/main/G050.2213-00.6063.csv

In [3]:
from astroquery.linelists.cdms import CDMS
import astropy.units as u
import astropy.constants as c
import peakutils
from peakutils.plot import plot as pplot
import matplotlib.pyplot as plt
import pandas as pd
import lineid_plot
from lmfit.models import GaussianModel
import numpy as np
from scipy.optimize import curve_fit

In [11]:
# search in CDMS database for plausible CH3OH lines in relevant range (see slides)
min_frequency=241.67
max_frequency=241.91
response = CDMS.query_lines(min_frequency= min_frequency* u.GHz,
                            max_frequency=max_frequency * u.GHz,
                            molecule="032504 CH3OH, vt=0-2",
                            get_query_payload=False,temperature_for_intensity=0)

In [ ]:
response

In [5]:
line_wave=response['FREQ']/1e3 # MHz to GHz
line_label=[str(i) for i in line_wave] # line label for plotting

In [6]:
fname='G050.2213-00.6063.csv'
df = pd.read_csv(fname)
freq=df['Freq'] # in GHz
i_nu=df['Intensity'] # in Jy sr^-1

In [ ]:
# plot cropped spectrum + plausible line locations
# peakutils is only used here as a visual aid

filter=(freq>min_frequency)&(freq<max_frequency)
freqn=freq[filter]
i_nun=i_nu[filter]

indices = peakutils.indexes(i_nun, thres=0.25, min_dist=0.1)  # indices are the "peak" positions
fig, ax = plt.subplots(figsize=(20,10),dpi=150)

pplot(freqn, i_nun, indices)
ax.set_xlim(min_frequency,max_frequency)
ax.set_ylim(-4e11,0.8e12)
lineid_plot.plot_line_ids(freqn, i_nun, line_wave, line_label, ax=ax, arrow_tip=0, box_loc=-3e11, max_iter=300, label1_size=6) # locate theoretical lines from cdms query
ax.set_xlabel('Frequency (GHz)')
ax.set_ylabel('Intensity (Jy sr^-1)')